# Data:

### The data selected for this project is the postal codes for Alberta, Canada. The codes below are how the data is proceed. Some criteria include:
### - No duplicated postal codes
### - Postal codes with no Borough is removed
### - Postal codes with no Neighborhood will have its Borough become the Neighborhood

### On top of these data, foursquare will also be used to obtain venue data in the project.

In [1]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import math

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'

In [10]:
wiki=requests.get(url).text
wiki_clean=soup(wiki, 'lxml')
wiki_clean

calgary=wiki_clean.find('table')

table_contents=[]

In [11]:
for row in calgary.findAll('td'):
    cell={}
    if row.span.text=='Not Assigned':
        pass
    else:
        cell['PostalCode']=row.b.text[:]
        cell['Borough']=(row.span.text).split('(')[0]
        try: 
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        except:
            pass
        table_contents.append(cell)


In [12]:
df=pd.DataFrame(table_contents)

In [13]:
df

,PostalCode,Borough,Neighborhood
0,T1A,Medicine HatCentral,NaN
1,T2A,Calgary,"Penbrooke Meadows, Marlborough"
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley"
3,T4A,AirdrieEast,NaN
4,T5A,Edmonton,"West Clareview, East Londonderry"
5,T6A,Edmonton,North Capilano
6,T7A,Drayton Valley,NaN
7,T8A,Sherwood ParkWest,NaN
8,T9A,Wetaskiwin,NaN
9,T1B,Medicine HatSouth,NaN


In [14]:
df.shape

(180, 3)

In [15]:
df.shape

(180, 3)

In [16]:
a=[]
for i in range(len(df['PostalCode'])):
    try:
        if 'Not assigned' in df[['Borough']].iloc[i].values or math.isnan(df[['PostalCode']].iloc[i].values[0]):# or 'Not assigned' in df[['Borough']].iloc[i].values[0]:
            a.append(i)
    except:
        pass
df.drop(df.index[a], inplace=True)
df.drop_duplicates(subset='PostalCode',inplace=True)

In [17]:
for i in range(len(df['PostalCode'])):
    try:
        if math.isnan(df[['Neighborhood']].iloc[i].values):
            df['Neighborhood'].iloc[i]=df['Borough'].iloc[i]
    except:
        pass

In [18]:
df

,PostalCode,Borough,Neighborhood
0,T1A,Medicine HatCentral,Medicine HatCentral
1,T2A,Calgary,"Penbrooke Meadows, Marlborough"
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley"
3,T4A,AirdrieEast,AirdrieEast
4,T5A,Edmonton,"West Clareview, East Londonderry"
5,T6A,Edmonton,North Capilano
6,T7A,Drayton Valley,Drayton Valley
7,T8A,Sherwood ParkWest,Sherwood ParkWest
8,T9A,Wetaskiwin,Wetaskiwin
9,T1B,Medicine HatSouth,Medicine HatSouth


In [20]:
df.to_csv('Calgary_clean.csv')

In [19]:
df.shape

(144, 3)

In [ ]:
#Data Cleaning is complete.
